In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval

In [30]:
data = pd.DataFrame(columns=["Model", "Idx", "Times"])
with open("hte_l_time.txt", "r") as f:
    for line in f.readlines():
        if line.startswith("Model"):
            continue
        d = line.split(";")
        d[2] = d[2].split("\n")[0]
        df = {"Model": str(d[0]), "Idx": int(d[1]), "Times": d[2]}
        data = data.append(pd.DataFrame(df, index=[0]))

data.reset_index(inplace=True)
data.drop(columns=["index"], inplace=True)

In [31]:
data.head()

,Model,Idx,Times
0,B1,0,"[(1, 147.99716329574585), (2, 131.089318037033..."
1,B1,1,"[(1, 138.3292236328125), (2, 132.6143598556518..."
2,B1,2,"[(1, 139.56730675697327), (2, 141.328600168228..."
3,B1,3,"[(1, 129.2668673992157), (2, 119.9269165992736..."
4,B1,4,"[(1, 141.2414722442627), (2, 127.1892259120941..."


In [32]:
data.dtypes

Model    object
Idx      object
Times    object
dtype: object

In [33]:
def times(row):
    d = literal_eval(row.Times)
    t = 0
    n = len(d)
    for i in d:
        t += i[1]
    return t/n

In [34]:
def epochs(row):
    d = literal_eval(row.Times)
    return len(d)

In [35]:
data["Avg"] = data.apply(lambda row: times(row), axis=1)
data["Epochs"] = data.apply(lambda row: epochs(row), axis=1)

In [36]:
data.head()

,Model,Idx,Times,Avg,Epochs
0,B1,0,"[(1, 147.99716329574585), (2, 131.089318037033...",134.473952,32
1,B1,1,"[(1, 138.3292236328125), (2, 132.6143598556518...",137.282973,21
2,B1,2,"[(1, 139.56730675697327), (2, 141.328600168228...",138.052156,21
3,B1,3,"[(1, 129.2668673992157), (2, 119.9269165992736...",128.592381,26
4,B1,4,"[(1, 141.2414722442627), (2, 127.1892259120941...",128.828348,22


In [37]:
def findAverageTime(df):
    avg = np.array(df.Avg)
    epo = np.array(df.Epochs)
    a = np.sum(avg*epo) / np.sum(epo)
    print(f"{df.Model.unique()[0]};{np.round(a, 2)}")
    return a

In [38]:
times = data.groupby(by=["Model"]).apply(findAverageTime)

A1;110.71
A2;115.14
A3;123.82
B1;133.3
B2;136.51
B3;181.23
C1;124.64
C2;143.73
C3;163.16
C4;170.04
D1;161.87
D2;200.54
D3;264.48


In [39]:
def findAverageEpochs(df):
    epo = np.array(df.Epochs)
    print(f"{df.Model.unique()[0]};{np.mean(epo)}")
    return np.mean(epo)

In [40]:
epochs = data.groupby(by=["Model"]).apply(findAverageEpochs)

A1;28.8
A2;26.8
A3;29.8
B1;24.4
B2;26.8
B3;25.2
C1;30.4
C2;28.0
C3;29.4
C4;25.2
D1;31.0
D2;27.8
D3;23.8


In [41]:
times*epochs

Model
A1    3188.578534
A2    3085.659153
A3    3689.766912
B1    3252.565944
B2    3658.546240
B3    4567.088594
C1    3789.174178
C2    4024.559622
C3    4796.937373
C4    4285.021913
D1    5018.115404
D2    5575.093849
D3    6294.710450
dtype: float64

In [42]:
params = pd.DataFrame(columns=["Model", "Params"])
with open("params.txt", "r") as f:
    for line in f.readlines():
        if line.startswith("Model"):
            continue
        d = line.split(";")
        d[1] = d[1].split("\n")[0]
        df = {"Model": str(d[0]), "Params": int(d[1])}
        params = params.append(pd.DataFrame(df, index=[0]))

params.reset_index(inplace=True)
params.drop(columns=["index"], inplace=True)

In [43]:
params.head()

,Model,Params
0,B1,55984
1,B2,121096
2,A1,178080
3,B3,278968
4,A2,356072


In [44]:
full = pd.merge(data, params, on=["Model"])

In [45]:
full.head()

,Model,Idx,Times,Avg,Epochs,Params
0,B1,0,"[(1, 147.99716329574585), (2, 131.089318037033...",134.473952,32,55984
1,B1,1,"[(1, 138.3292236328125), (2, 132.6143598556518...",137.282973,21,55984
2,B1,2,"[(1, 139.56730675697327), (2, 141.328600168228...",138.052156,21,55984
3,B1,3,"[(1, 129.2668673992157), (2, 119.9269165992736...",128.592381,26,55984
4,B1,4,"[(1, 141.2414722442627), (2, 127.1892259120941...",128.828348,22,55984


In [46]:
def timePerParam(df):
    avg = np.array(df.Avg)
    epo = np.array(df.Epochs)
    par = np.mean(df.Params)
    a = np.sum(avg*epo) / np.sum(epo)
    print(f"{df.Model.unique()[0]};{np.around(a, 2)};{par}")

In [47]:
full.groupby(by=["Model"]).apply(timePerParam)

A1;110.71;178080.0
A2;115.14;356072.0
A3;123.82;534064.0
B1;133.3;55984.0
B2;136.51;121096.0
B3;181.23;278968.0
C1;124.64;1056232.0
C2;143.73;2358448.0
C3;163.16;3133176.0
C4;170.04;4176760.0
D1;161.87;741088.0
D2;200.54;1301432.0
D3;264.48;1956384.0


""
